In [1]:
# import
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
import os

# label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================

def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type   

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            if label != "":
                dataset.append(
                                {   
                                    'uuid': root.split("\\")[-1],
                                    'label': label,
                                    'image_path': os.path.join(root, file)
                                }
                            )

    return dataset


In [2]:
## 參數設置
image_dir = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"
concat_type = "channel"
class_count = 3

In [ ]:
## image spatail
# =========================
from PIL import Image
import os

def resize_and_merge_images(image_a_path, image_b_path, output_path):
    try:
        with Image.open(image_a_path) as img_a, Image.open(image_b_path) as img_b:
            img_a = img_a.resize((224, 224))
            img_b = img_b.resize((224, 224))

            # Convert images to numpy arrays
            img_a_array = np.array(img_a)
            img_b_array = np.array(img_b)

            # Extract first layers of A and B
            c_layer_1 = img_a_array[:, :, 0]  # First channel of A
            c_layer_3 = img_b_array[:, :, 0]  # First channel of B

            # 確定三層長一樣
            # print(np.array_equal(img_a_array[:, :, 0], img_a_array[:, :, 1]))
            # print(np.array_equal(img_b_array[:, :, 0], img_b_array[:, :, 1]))
            
            # Compute second layer as (A first layer + B first layer) / 2
            c_layer_2 = ((img_a_array[:, :, 0].astype(np.float32) + img_b_array[:, :, 0].astype(np.float32)) / 2).astype(np.uint8)
            
            # Stack layers to create new image
            merged_image_array = np.stack([c_layer_1, c_layer_2, c_layer_3], axis=2)
            merged_image = Image.fromarray(merged_image_array, mode='RGB')
            
            # Save the merged image
            merged_image.save(output_path)
            # print(f"Merged image saved at {output_path}")
    except Exception as e:
        print(f"Error processing images: {e}")

# Example usage
# a_image_path = "path/to/a_image.jpg"
# b_image_path = "path/to/b_image.jpg"
# output_image_path = "path/to/output.jpg"

# resize_and_merge_images(a_image_path, b_image_path, output_image_path)

# =========================

In [ ]:
## load data and  labels
# =========================
data = load_path(image_dir, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images_front = pd.concat([filepaths, labels], axis=1)
# =========================

## merge and save image
# =========================
images_side = images_front.copy()
images_side.loc[:, "Filepath"] = images_front["Filepath"].str.replace("front", "side")
images_side.loc[:, "Filepath"] = images_front["Filepath"].str.replace("front", "side")
for i in range(len(images_front)):
    resize_and_merge_images(images_front.loc[i]["Filepath"], images_side.loc[i]["Filepath"], images_front.loc[i]["Filepath"].replace("front", "channel"))
# =========================

In [5]:
## load channel and split
image_dir2 = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\channel"
data = load_path(image_dir2, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images_channel = pd.concat([filepaths, labels], axis=1)
train_df, test_df = train_test_split(images_channel, train_size=0.8, shuffle=True, random_state=44, stratify=images_channel['Label'])

print("Training set label distribution:\n", train_df['Label'].value_counts(normalize=False))
print("Test set label distribution:\n", test_df['Label'].value_counts(normalize=False))

# 關閉翻轉
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
                                                                    validation_split=0.2)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
# =========================

Training set label distribution:
 0    128
2     95
1     93
Name: Label, dtype: int64
Test set label distribution:
 0    32
1    24
2    24
Name: Label, dtype: int64


In [6]:

# load model
# =========================
pretrained_model = tf.keras.applications.resnet50.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg')

pretrained_model.trainable = False

inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
model = tf.keras.Model(inputs, outputs)
# print(model.summary())
# =========================

## 分資料
# =========================

# 確認
print(train_df.iloc[0]['Filepath'].split("\\")[-3])

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


## compile and evaluate
# =========================

print("-------Training " + "_" + concat_type + "-------")
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

## early stop 
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# history=model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
## no early stop
history = model.fit(train_images, validation_data=val_images, epochs=30)

results = model.evaluate(test_images, verbose=0)
# =========================


## save model to this path
# =========================
model.save("./weights/"+concat_type+"_" + "_frac.h5")
# =========================


## print results
# =========================
# print(save_path + "_" + concat_type + "_Results:")
pred = model.predict(test_images)
predicted_labels = np.argmax(pred, axis=1)

# f1 = f1_score(test_images.labels, predicted_labels, average='macro')
# precision = precision_score(test_images.labels, predicted_labels, average='macro')
# recall = recall_score(test_images.labels, predicted_labels, average='macro')

# print(results)
# print(f"Test Accuracy: {np.round(results[1], 2)}")
# print(f"f1 score: {np.round(f1, 2)}")
# print(f"precision: {np.round(precision, 2)}")
# print(f"recall: {np.round(recall, 2)}")
# =========================


# create plots for accuracy and save it
# =========================
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_Accuracy.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## create plots for loss and save it
# =========================
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Loss.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## plot confusion matrix
# =========================
if class_count == 2:
    display_labels = [0, 1]
elif class_count == 3:
    display_labels = [0, 1, 2]
elif class_count == 4:
    display_labels = [0, 1, 2, 3]


cm = confusion_matrix(test_images.labels, predicted_labels)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = display_labels)
cm_display.plot()
plt.title('Confusion Matrix')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Confusion Matrix.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================

channel
Found 253 validated image filenames belonging to 3 classes.
Found 63 validated image filenames belonging to 3 classes.
Found 80 validated image filenames belonging to 3 classes.
-------Training _channel-------
Epoch 1/30
4/4 [==============================] - 6s 741ms/step - loss: 1.2768 - accuracy: 0.3123 - val_loss: 1.0983 - val_accuracy: 0.4127
Epoch 2/30
4/4 [==============================] - 1s 124ms/step - loss: 1.0751 - accuracy: 0.4150 - val_loss: 1.0565 - val_accuracy: 0.4762
Epoch 3/30
4/4 [==============================] - 0s 120ms/step - loss: 1.0452 - accuracy: 0.4743 - val_loss: 1.0169 - val_accuracy: 0.4762
Epoch 4/30
4/4 [==============================] - 0s 119ms/step - loss: 0.9812 - accuracy: 0.5020 - val_loss: 0.9475 - val_accuracy: 0.5873
Epoch 5/30
4/4 [==============================] - 1s 124ms/step - loss: 0.9101 - accuracy: 0.6126 - val_loss: 0.8793 - val_accuracy: 0.6349
Epoch 6/30
4/4 [==============================] - 1s 134ms/step - loss: 0.8550 - a

d:\anaconda3\envs\bone_20240719\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [7]:
acc = accuracy_score(test_images.labels, predicted_labels)
f1 = f1_score(test_images.labels, predicted_labels, average='macro')
precision = precision_score(test_images.labels, predicted_labels, average='macro')
recall = recall_score(test_images.labels, predicted_labels, average='macro')

print(f"Test Accuracy: {np.round(acc, 2)}")
print(f"f1 score: {np.round(f1, 2)}")
print(f"precision: {np.round(precision, 2)}")
print(f"recall: {np.round(recall, 2)}")

Test Accuracy: 0.72
f1 score: 0.69
precision: 0.69
recall: 0.69
